# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [264]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [265]:
output_weather_file = "../WeatherPy/output_data/city_weather.csv"
weather_data = pd.read_csv(output_weather_file)

In [268]:
weather_data.head()

,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds
0,Tiksi,RU,71.64,128.87,25.62,25.62,83,21.00,96
1,Vaini,IN,15.34,74.49,83.46,84.20,65,4.70,40
2,Erdenet,MN,49.03,104.04,56.33,56.33,34,15.73,0
3,Ushuaia,AR,-54.81,-68.31,32.00,32.00,93,38.03,90
4,Mataura,NZ,-46.19,168.86,51.01,51.01,94,1.99,100


In [269]:
len(weather_data)

552

In [270]:
# Store latitude and longitude in locations
locations = weather_data[["lat", "long"]]

# Fill NaN values and convert to float
humidity = weather_data["humidity"].astype(float)

In [271]:
len(locations)

552

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [272]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [274]:
# Narrow down the DataFrame to find your ideal weather condition. 
# Drop a row by condition  
# A max temperature >75F and <=80F lower than 80 degrees but higher than 75.
# very few clouds (<5%) cloudiness.
# Wind speed < 10 (light breeze)


myCities = weather_data.loc[weather_data["temp"]>75]
myCities = myCities.loc[myCities["temp"]<80]
myCities = myCities.loc[myCities["wind speed"]<10]
myCities = myCities.loc[myCities["clouds"]<5]



In [275]:
#myCities.count()
myCities

,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds
20,Georgetown,GY,6.80,-58.16,75.20,75.20,100,4.70,0
61,Nieuw Amsterdam,SR,5.88,-55.08,77.29,78.01,94,4.70,0
219,Morondava,MG,-20.30,44.28,78.78,78.78,75,8.10,0
281,Abaetetuba,BR,-1.72,-48.88,77.00,77.00,88,9.17,0
410,Gweta,BW,-20.21,25.26,77.14,77.14,17,7.11,0
442,Dasoguz,TM,41.84,59.97,76.10,77.00,27,6.93,0
456,Taoudenni,ML,22.68,-3.98,79.79,79.79,14,8.39,0
482,Chingola,ZM,-12.55,27.86,75.03,75.03,45,8.37,4


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [276]:
hotel_df = myCities
hotel_df

,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds
20,Georgetown,GY,6.80,-58.16,75.20,75.20,100,4.70,0
61,Nieuw Amsterdam,SR,5.88,-55.08,77.29,78.01,94,4.70,0
219,Morondava,MG,-20.30,44.28,78.78,78.78,75,8.10,0
281,Abaetetuba,BR,-1.72,-48.88,77.00,77.00,88,9.17,0
410,Gweta,BW,-20.21,25.26,77.14,77.14,17,7.11,0
442,Dasoguz,TM,41.84,59.97,76.10,77.00,27,6.93,0
456,Taoudenni,ML,22.68,-3.98,79.79,79.79,14,8.39,0
482,Chingola,ZM,-12.55,27.86,75.03,75.03,45,8.37,4


In [277]:
hotel_df["hotel"]=" "

In [278]:
hotel_df["city"]

20          Georgetown
61     Nieuw Amsterdam
219          Morondava
281         Abaetetuba
410              Gweta
442            Dasoguz
456          Taoudenni
482           Chingola
Name: city, dtype: object

In [279]:
hotel_df.reset_index()

,index,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds,hotel
0,20,Georgetown,GY,6.80,-58.16,75.20,75.20,100,4.70,0,
1,61,Nieuw Amsterdam,SR,5.88,-55.08,77.29,78.01,94,4.70,0,
2,219,Morondava,MG,-20.30,44.28,78.78,78.78,75,8.10,0,
3,281,Abaetetuba,BR,-1.72,-48.88,77.00,77.00,88,9.17,0,
4,410,Gweta,BW,-20.21,25.26,77.14,77.14,17,7.11,0,
5,442,Dasoguz,TM,41.84,59.97,76.10,77.00,27,6.93,0,
6,456,Taoudenni,ML,22.68,-3.98,79.79,79.79,14,8.39,0,
7,482,Chingola,ZM,-12.55,27.86,75.03,75.03,45,8.37,4,


In [280]:
# find the closest hotel of each type to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
radius = 5000
params = {
    "location": "",  # philadelphia coords,
    "rankby": "distance",
    "keyword": "hotel",
    "type": "hotel",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    params["location"] = f"{hotel_df.loc[index, 'lat']},{hotel_df.loc[index, 'long']}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()

        
    # extract results
    results = response['results']
    


    try:
        hotel_df.loc[index, 'hotel'] = results[myIndex]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

------------
------------
------------
------------
------------
------------
Missing field/result... skipping.
------------
------------


In [281]:
hotel_df

,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds,hotel
20,Georgetown,GY,6.80,-58.16,75.20,75.20,100,4.70,0,Hotel Princess Raven
61,Nieuw Amsterdam,SR,5.88,-55.08,77.29,78.01,94,4.70,0,d' ARK (d' Jazz ARK)
219,Morondava,MG,-20.30,44.28,78.78,78.78,75,8.10,0,SELECT HOTEL
281,Abaetetuba,BR,-1.72,-48.88,77.00,77.00,88,9.17,0,Hotel Santiago
410,Gweta,BW,-20.21,25.26,77.14,77.14,17,7.11,0,Gweta Lodge
442,Dasoguz,TM,41.84,59.97,76.10,77.00,27,6.93,0,Hotel Şabat
456,Taoudenni,ML,22.68,-3.98,79.79,79.79,14,8.39,0,
482,Chingola,ZM,-12.55,27.86,75.03,75.03,45,8.37,4,Mokorro Hotel


In [288]:
hotel_df=hotel_df[hotel_df["hotel"]!=" "]
hotel_df

,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds,hotel
20,Georgetown,GY,6.80,-58.16,75.20,75.20,100,4.70,0,Hotel Princess Raven
61,Nieuw Amsterdam,SR,5.88,-55.08,77.29,78.01,94,4.70,0,d' ARK (d' Jazz ARK)
219,Morondava,MG,-20.30,44.28,78.78,78.78,75,8.10,0,SELECT HOTEL
281,Abaetetuba,BR,-1.72,-48.88,77.00,77.00,88,9.17,0,Hotel Santiago
410,Gweta,BW,-20.21,25.26,77.14,77.14,17,7.11,0,Gweta Lodge
442,Dasoguz,TM,41.84,59.97,76.10,77.00,27,6.93,0,Hotel Şabat
482,Chingola,ZM,-12.55,27.86,75.03,75.03,45,8.37,4,Mokorro Hotel


In [289]:
narrowed_city_df = hotel_df[["hotel","city","country","lat","long"]]
narrowed_city_df=narrowed_city_df.rename(columns={"hotel": "Hotel Name","city": "City","country":"Country","lat":"Lat","long":"Lng"})
narrowed_city_df.head()

,Hotel Name,City,Country,Lat,Lng
20,Hotel Princess Raven,Georgetown,GY,6.80,-58.16
61,d' ARK (d' Jazz ARK),Nieuw Amsterdam,SR,5.88,-55.08
219,SELECT HOTEL,Morondava,MG,-20.30,44.28
281,Hotel Santiago,Abaetetuba,BR,-1.72,-48.88
410,Gweta Lodge,Gweta,BW,-20.21,25.26


In [290]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [291]:
len(narrowed_city_df)

7

In [292]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)


# Add the layer to the map
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…